A rough copy of https://jaketae.github.io/study/pytorch-rnn/

In [1]:
%set_env TORCH=1

env: TORCH=1


In [2]:
import random
from io import BytesIO
from pathlib import Path
from zipfile import ZipFile
from string import ascii_letters
import requests
import numpy as np
from unidecode import unidecode
from tinygrad.tensor import Tensor
from tinygrad.nn import optim
from extra.training import sparse_categorical_crossentropy
from sklearn.model_selection import train_test_split
random.seed(1337)
Tensor.manual_seed(1337)

In [3]:
names_dir = Path("..").resolve() / "data" / "names"
data_dir = names_dir / "data" / "names"

In [4]:
if not data_dir.is_dir():
    names_dir.mkdir(parents=True, exist_ok=True)
    response = requests.get("https://download.pytorch.org/tutorial/data.zip")
    assert response.status_code == 200
    zip_file = BytesIO(response.content)
    with ZipFile(zip_file, "r") as zip_ref:
        zip_ref.extractall(names_dir)

In [5]:
lang2label = {file_path.stem: i for i, file_path in enumerate(data_dir.iterdir())}
num_langs = len(lang2label)

In [6]:
char2idx = {letter: i for i, letter in enumerate(ascii_letters + " .,:;-'")}
num_letters = len(char2idx)

In [7]:
def name2array(name: str) -> Tensor:
    array = np.zeros((len(name), 1, num_letters))
    for i, char in enumerate(name):
        array[i][0][char2idx[char]] = 1
    return array

In [8]:
input_names, target_langs = [], []

for file_path in data_dir.iterdir():
    with file_path.open("r") as file:
        for name in [unidecode(line.rstrip()) for line in file]:
            if all(letter in char2idx for letter in name):
                input_names.append(name2array(name))            
                target_langs.append(lang2label[file_path.stem])

In [9]:
train_idx, test_idx = train_test_split(range(len(target_langs)), test_size=0.1, shuffle=True, stratify=target_langs)
train_dataset = [(Tensor(input_names[i]), np.array(target_langs[i])) for i in train_idx]
test_dataset = [(Tensor(input_names[i]), np.array(target_langs[i])) for i in test_idx]

In [10]:
class RecurrentNet:
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        self.hidden_size = hidden_size
        self.in2hidden = Tensor.scaled_uniform(input_size + hidden_size, hidden_size)
        self.in2output = Tensor.scaled_uniform(input_size + hidden_size, output_size)
    
    def __call__(self, x: Tensor, hidden_state: Tensor) -> Tensor:
        x = Tensor.cat(x, hidden_state, dim=1)
        hidden = x.dot(self.in2hidden).sigmoid()
        output = x.dot(self.in2output).log_softmax()
        return output, hidden
    
    def init_hidden(self):
        hidden = Tensor.zeros(1, self.hidden_size)
        return hidden

In [11]:
model = RecurrentNet(num_letters, 256, num_langs)
criterion = sparse_categorical_crossentropy
optimizer = optim.Adam(optim.get_parameters(model), lr=0.001)

In [12]:
num_epochs = 2
plot_interval = 6000

for epoch in range(num_epochs):
    random.shuffle(train_dataset)
    for i, (name, label) in enumerate(train_dataset):
        hidden_state = model.init_hidden()
        for char in name:
            output, hidden_state = model(char, hidden_state)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % plot_interval == 0:
            print(f"epoch {epoch+1}/{num_epochs} step {i+1}/{len(train_dataset)} loss {loss.numpy()[0]:2f}")

epoch 1/2 step 6000/18063 loss 0.011229
epoch 1/2 step 12000/18063 loss 0.021270
epoch 1/2 step 18000/18063 loss 2.832460
epoch 2/2 step 6000/18063 loss 0.451736
epoch 2/2 step 12000/18063 loss 0.000200
epoch 2/2 step 18000/18063 loss 0.435729


In [13]:
num_correct = 0

for name, label in test_dataset:
    hidden_state = model.init_hidden()
    for char in name:
        output, hidden_state = model(char, hidden_state)
    pred = output.cpu().numpy().argmax()
    num_correct += (pred == label).sum()

print(f"test set accuracy is {num_correct / len(test_dataset) * 100:.4f}%")

test set accuracy is 73.1938%


In [14]:
label2lang = {label: lang for lang, label in lang2label.items()}

def predict(name):
    tensor_name = Tensor(name2array(name))
    hidden_state = model.init_hidden()
    for char in tensor_name:
        output, hidden_state = model(char, hidden_state)
    pred = output.cpu().numpy().argmax()
    return label2lang[pred]

In [15]:
predict("Jake")

'English'

In [16]:
predict("Qin")

'Korean'

In [17]:
predict("Fernando")

'Italian'

In [18]:
predict("Demirkan")

'Russian'